In [ ]:
# Copyright : Ryoungseob Kwon
# Cite this : Enabling 30 m maize mapping without ground reference leveraging multiple satellite platforms in Brazil and Argentina

import ee
ee.Authenticate()
ee.Initialize()

!pip install geemap -q
import geemap

import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Train area
input_train_dir = '/content/drive/MyDrive/download_GEDI/shapefiles'
input_train = os.path.join(input_train_dir, 'USA_train_area.shp')
input_train_ee = geemap.shp_to_ee(input_train)

roi = ee.FeatureCollection.geometry(input_train_ee)
roi_Map = ee.Image().byte().paint(**{'featureCollection': roi, 'color': 'red', 'width': 2})

# Test area
input_test_ee = []
for i in range(10):
  fname = 'test_point_' + str(i+1).zfill(2) + '.shp'
  test_point = geemap.shp_to_ee(os.path.join(input_train_dir, fname))
  ee_test_point = ee.FeatureCollection.geometry(test_point)
  ee_test_buffer = ee_test_point.buffer(100000)
  input_test_ee.append(ee_test_buffer)

# Load datasets
begin_date = ee.String('2021-01-01')
end_date   = ee.String('2021-12-31')

cdl = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(begin_date, end_date)).first()
cropLandcover = cdl.select('cropland')
cdl_composite = cropLandcover.reduce(ee.Reducer.mode())

# Crop class masking
maize = cdl_composite.eq(1)     # corn
others = cdl_composite.neq(1)   # other crops + landcover

# Cropland Masking
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
target_dw_images = dw.filterDate(begin_date, end_date)
classification = target_dw_images.select("label")
dwComposite = classification.reduce(ee.Reducer.mode())
cropland_lc = dwComposite.eq(4)

maize_cropland_masked = maize.updateMask(cropland_lc).selfMask()
others_cropland_masked = others.updateMask(cropland_lc).selfMask()

maize_masked = maize_cropland_masked.clip(roi)
others_masked = others_cropland_masked.clip(roi)

# GEDI Functions
def qualityMask(im):
  im_masked = im.updateMask(im.select('quality_flag').eq(1)).updateMask(im.select('degrade_flag').eq(0)).updateMask(im.select('rh100').lte(10))
  return im_masked

def sensitivity_filter(img):
  img_sens = img.updateMask(img.select('sensitivity').gte(0.95))
  return img_sens

# Sentinel-2 Functions
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = (1 << 10)
  cirrusBitMask = (1 << 11)
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  cloudless = image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])
  return cloudless

def add_ndvi(image):
  ndvi_image = image.expression('(float(b("B8") - b("B4")) / float(b("B8") + b("B4")))').rename('NDVI').copyProperties(image, ["system:time_start"])
  add_image = image.addBands(ndvi_image)
  return add_image

def s2_scale(image):
  scaled = image.multiply(0.0001).copyProperties(image, ["system:time_start"])
  return scaled

# Train datasets Functions
def create_feature(i):
  return ee.Feature(i.geometry(), {'id': i.id()})

In [ ]:
number_of_points = ee.Number(100000)

**Download GEDI paths first, for evaluation**

In [ ]:
gedi = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filter(ee.Filter.date(begin_date, end_date)).filter(ee.Filter.calendarRange(8, 8, 'month')).map(qualityMask).map(sensitivity_filter).select('rh.*')
gedi_mean = gedi.mean()
gedi_mask = gedi_mean.select('rh98').gt(0)

gedi_path = gedi_mask.updateMask(cropland_lc)
gedi_path_output_Int = gedi_path.toInt16()

for i in range(len(input_test_ee)):
  print('task ' + str(i+1) + 'processing...')
  test_roi = input_test_ee[i]
  fname = 'gedi_path_' + str(i+1).zfill(2)

  task = ee.batch.Export.image.toDrive(
    image = gedi_path_output_Int,
    fileFormat = 'GeoTIFF',
    folder = 'gedi_path',
    description = fname,
    region = test_roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
  )

  task.start()

# **GEDI 101 Bands**

In [ ]:
gedi8 = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filter(ee.Filter.date(begin_date, end_date)).filter(ee.Filter.calendarRange(8, 8, 'month')).map(qualityMask).map(sensitivity_filter).select('rh.*')
gedi8_mean = gedi8.mean()
gedi8_mask = gedi8_mean.select('rh98').gt(0)

maize_gedi8 = maize_masked.updateMask(gedi8_mask)
others_gedi8 = others_masked.updateMask(gedi8_mask)

maize_samples    = maize_gedi8.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
maize_geometry   = maize_samples.geometry()
maize_geometries = maize_geometry.geometries()
maize_train      = ee.Algorithms.GeometryConstructors.MultiPoint(maize_geometries, None)
maize_for_train  = ee.FeatureCollection([ee.Feature((maize_train), {"height": 0})])

others_samples    = others_gedi8.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
others_geometry   = others_samples.geometry()
others_geometries = others_geometry.geometries()
others_train      = ee.Algorithms.GeometryConstructors.MultiPoint(others_geometries, None)
others_for_train  = ee.FeatureCollection([ee.Feature((others_train), {"height": 1})])

train8 = maize_for_train.merge(others_for_train)

train_pb8 = gedi8_mean.sampleRegions(**{
  'collection': train8,
  'properties': ['height'],
  'scale': 25,
  'tileScale': 16
})

classifier_pb8 = ee.Classifier.smileRandomForest(**{'numberOfTrees': 40})
trained_pb8 = classifier_pb8.train(**{
  'features': train_pb8,
  'classProperty': 'height',
  'inputProperties': gedi8_mean.bandNames()
})

gedi_101_classified = gedi8_mean.classify(trained_pb8)
gedi_101_results = gedi_101_classified.updateMask(cropland_lc)
image1 = ee.Image.constant(1)
gedi_101_output = gedi_101_results.add(image1)
gedi_101_output_Int = gedi_101_output.toInt16()

for i in range(len(input_test_ee)):
  print('task ' + str(i+1) + 'processing...')
  test_roi = input_test_ee[i]
  fname = 'gedi_101_' + str(i+1).zfill(2)

  task = ee.batch.Export.image.toDrive(
    image = gedi_101_output_Int,
    fileFormat = 'GeoTIFF',
    folder = 'gedi_101_classified',
    description = fname,
    region = test_roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
  )

  task.start()


# **GEDI 11 Bands**

In [ ]:
gedi = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filter(ee.Filter.date(begin_date, end_date)).filter(ee.Filter.calendarRange(8, 8, 'month')).map(qualityMask).map(sensitivity_filter).select('rh.*')
gedi_mean = gedi.mean()

trainband_01 = gedi_mean.select('rh0')
trainband_02 = gedi_mean.select('rh100')
trainband_03 = gedi_mean.select('rh2')
trainband_04 = gedi_mean.select('rh1')
trainband_05 = gedi_mean.select('rh3')
trainband_06 = gedi_mean.select('rh99')
trainband_07 = gedi_mean.select('rh5')
trainband_08 = gedi_mean.select('rh4')
trainband_09 = gedi_mean.select('rh96')
trainband_10 = gedi_mean.select('rh7')
trainband_11 = gedi_mean.select('rh98')

gedi_trainbands = trainband_01.addBands(trainband_02).addBands(trainband_03).addBands(trainband_04).addBands(trainband_05).addBands(trainband_06).addBands(trainband_07).addBands(trainband_08).addBands(trainband_09).addBands(trainband_10).addBands(trainband_11)
gedi_mask = gedi_mean.select('rh98').gt(0)
gedi_trainbands_masked = gedi_trainbands.updateMask(gedi_mask)

maize_gedi = maize_masked.updateMask(gedi_mask)
others_gedi = others_masked.updateMask(gedi_mask)

maize_samples    = maize_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
maize_geometry   = maize_samples.geometry()
maize_geometries = maize_geometry.geometries()
maize_train      = ee.Algorithms.GeometryConstructors.MultiPoint(maize_geometries, None)
maize_for_train  = ee.FeatureCollection([ee.Feature((maize_train), {"height": 0})])

others_samples    = others_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
others_geometry   = others_samples.geometry()
others_geometries = others_geometry.geometries()
others_train      = ee.Algorithms.GeometryConstructors.MultiPoint(others_geometries, None)
others_for_train  = ee.FeatureCollection([ee.Feature((others_train), {"height": 1})])

train = maize_for_train.merge(others_for_train)

train_pb = gedi_trainbands.sampleRegions(**{
  'collection': train,
  'properties': ['height'],
  'scale': 25,
  'tileScale': 16
})

classifier_pb = ee.Classifier.smileRandomForest(**{'numberOfTrees': 40})
trained_pb = classifier_pb.train(**{
  'features': train_pb,
  'classProperty': 'height',
  'inputProperties': gedi_trainbands.bandNames()
})

gedi_11_classified = gedi_mean.classify(trained_pb)
gedi_11_results = gedi_11_classified.updateMask(cropland_lc)
image1 = ee.Image.constant(1)
gedi_11_output = gedi_11_results.add(image1)
gedi_11_output_Int = gedi_11_output.toInt16()

for i in range(len(input_test_ee)):
  print('task ' + str(i+1) + 'processing...')
  test_roi = input_test_ee[i]
  fname = 'gedi_11_' + str(i+1).zfill(2)

  task = ee.batch.Export.image.toDrive(
    image = gedi_11_output_Int,
    fileFormat = 'GeoTIFF',
    folder = 'gedi_11_classified',
    description = fname,
    region = test_roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
  )

  task.start()

# **Sentinel-2 only**

In [ ]:
s2col_sow   = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-04-01', '2021-05-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 65)).map(maskS2clouds)
s2col_grow  = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-06-01', '2021-08-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 65)).map(maskS2clouds)
s2col_harv  = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-09-01', '2021-11-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 65)).map(maskS2clouds)

s2_sow_blue  = s2col_sow.map(s2_scale).median().select('B2').rename('B2_sow')
s2_sow_green = s2col_sow.map(s2_scale).median().select('B3').rename('B3_sow')
s2_sow_red   = s2col_sow.map(s2_scale).median().select('B4').rename('B4_sow')
s2_sow_nir   = s2col_sow.map(s2_scale).median().select('B8').rename('B8_sow')
s2_sow_swir1 = s2col_sow.map(s2_scale).median().select('B11').rename('B11_sow')
s2_sow_swir2 = s2col_sow.map(s2_scale).median().select('B12').rename('B12_sow')
s2_sow_ndvi  = s2col_sow.map(add_ndvi).median().select('NDVI').rename('NDVI_sow')

s2_grow_blue  = s2col_grow.map(s2_scale).median().select('B2').rename('B2_grow')
s2_grow_green = s2col_grow.map(s2_scale).median().select('B3').rename('B3_grow')
s2_grow_red   = s2col_grow.map(s2_scale).median().select('B4').rename('B4_grow')
s2_grow_nir   = s2col_grow.map(s2_scale).median().select('B8').rename('B8_grow')
s2_grow_swir1 = s2col_grow.map(s2_scale).median().select('B11').rename('B11_grow')
s2_grow_swir2 = s2col_grow.map(s2_scale).median().select('B12').rename('B12_grow')
s2_grow_ndvi  = s2col_grow.map(add_ndvi).median().select('NDVI').rename('NDVI_grow')

s2_harv_blue  = s2col_harv.map(s2_scale).median().select('B2').rename('B2_harv')
s2_harv_green = s2col_harv.map(s2_scale).median().select('B3').rename('B3_harv')
s2_harv_red   = s2col_harv.map(s2_scale).median().select('B4').rename('B4_harv')
s2_harv_nir   = s2col_harv.map(s2_scale).median().select('B8').rename('B8_harv')
s2_harv_swir1 = s2col_harv.map(s2_scale).median().select('B11').rename('B11_harv')
s2_harv_swir2 = s2col_harv.map(s2_scale).median().select('B12').rename('B12_harv')
s2_harv_ndvi  = s2col_harv.map(add_ndvi).median().select('NDVI').rename('NDVI_harv')

s2_trainbands = s2_sow_blue.addBands(s2_sow_green).addBands(s2_sow_red).addBands(s2_sow_nir).addBands(s2_sow_swir1).addBands(s2_sow_swir2).addBands(s2_sow_ndvi).addBands(s2_grow_blue).addBands(s2_grow_green).addBands(s2_grow_red).addBands(s2_grow_nir).addBands(s2_grow_swir1).addBands(s2_grow_swir2).addBands(s2_grow_ndvi).addBands(s2_harv_blue).addBands(s2_harv_green).addBands(s2_harv_red).addBands(s2_harv_nir).addBands(s2_harv_swir1).addBands(s2_harv_swir2).addBands(s2_harv_ndvi)
s2_trainbands_masked = s2_trainbands.updateMask(gedi_mask)

maize_gedi = maize_masked.updateMask(gedi_mask)
others_gedi = others_masked.updateMask(gedi_mask)

maize_samples    = maize_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
maize_geometry   = maize_samples.geometry()
maize_geometries = maize_geometry.geometries()
maize_train      = ee.Algorithms.GeometryConstructors.MultiPoint(maize_geometries, None)
maize_for_train  = ee.FeatureCollection([ee.Feature((maize_train), {"height": 0})])

others_samples    = others_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
others_geometry   = others_samples.geometry()
others_geometries = others_geometry.geometries()
others_train      = ee.Algorithms.GeometryConstructors.MultiPoint(others_geometries, None)
others_for_train  = ee.FeatureCollection([ee.Feature((others_train), {"height": 1})])

train = maize_for_train.merge(others_for_train)

train_pb_s2_only = s2_trainbands_masked.sampleRegions(**{
  'collection': train,
  'properties': ['height'],
  'scale': 25,
  'tileScale': 16
})

classifier_pb_s2_only = ee.Classifier.smileRandomForest(**{'numberOfTrees': 40})
trained_pb_s2_only = classifier_pb_s2_only.train(**{
  'features': train_pb_s2_only,
  'classProperty': 'height',
  'inputProperties': s2_trainbands_masked.bandNames()
})

sen2_classified = s2_trainbands_masked.classify(trained_pb_s2_only)
sen2_results = sen2_classified.updateMask(cropland_lc)
image1 = ee.Image.constant(1)
sen2_output = sen2_results.add(image1)
sen2_output_Int = sen2_output.toInt16()

for i in range(len(input_test_ee)):
  print('task ' + str(i+1) + 'processing...')
  test_roi = input_test_ee[i]
  fname = 'sen2_' + str(i+1).zfill(2)

  task = ee.batch.Export.image.toDrive(
    image = sen2_output_Int,
    fileFormat = 'GeoTIFF',
    folder = 'sen2_classified',
    description = fname,
    region = test_roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
  )

  task.start()

# **GEDI + Sentinel-2**

In [ ]:
gedi_s2 = gedi_trainbands_masked.addBands(s2_trainbands_masked)

maize_gedi  = maize_masked.updateMask(gedi_mask)
others_gedi = others_masked.updateMask(gedi_mask)

maize_samples    = maize_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
maize_geometry   = maize_samples.geometry()
maize_geometries = maize_geometry.geometries()
maize_train      = ee.Algorithms.GeometryConstructors.MultiPoint(maize_geometries, None)
maize_for_train  = ee.FeatureCollection([ee.Feature((maize_train), {"height": 0})])

others_samples    = others_gedi.stratifiedSample(**{'numPoints':number_of_points, 'scale':25, 'region':roi, 'geometries': True}).map(create_feature)
others_geometry   = others_samples.geometry()
others_geometries = others_geometry.geometries()
others_train      = ee.Algorithms.GeometryConstructors.MultiPoint(others_geometries, None)
others_for_train  = ee.FeatureCollection([ee.Feature((others_train), {"height": 1})])

train = maize_for_train.merge(others_for_train)

train_pb_all = gedi_s2.sampleRegions(**{
  'collection': train,
  'properties': ['height'],
  'scale': 25,
  'tileScale': 16
})

classifier_pb_all = ee.Classifier.smileRandomForest(**{'numberOfTrees': 40})
trained_pb_all = classifier_pb_all.train(**{
  'features': train_pb_all,
  'classProperty': 'height',
  'inputProperties': gedi_s2.bandNames()
})

all_classified = gedi_s2.classify(trained_pb_all)
all_results = all_classified.updateMask(cropland_lc)
image1 = ee.Image.constant(1)
all_output = all_results.add(image1)
all_output_Int = all_output.toInt16()

for i in range(len(input_test_ee)):
  print('task ' + str(i+1) + 'processing...')
  test_roi = input_test_ee[i]
  fname = 'gedi_sen2_' + str(i+1).zfill(2)

  task = ee.batch.Export.image.toDrive(
    image = all_output_Int,
    fileFormat = 'GeoTIFF',
    folder = 'gedi_sen2_classified',
    description = fname,
    region = test_roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
  )

  task.start()